In [1]:
## personal imports
from processor.processor import Processor as p
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from functional.backtester import Backtester as b
from transformer.adhoc_transforms import Adhoc as adhocs
from transformer.risk_transforms import Risk as risks


## additional imports
from statistics import variance
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm

In [2]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [3]:
market = Market()
speculation_db = ADatabase("spec")
classification_db = ADatabase("spec_classification")
class_spec = ADatabase("class_spec")
weeklies_db = ADatabase("weeklies")

In [4]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [5]:
tyields = adhocs.tyields()
bench_returns = adhocs.spy_bench()

In [6]:
new_prices = []
market.connect()
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        ticker_sim = market.retrieve_ticker_prices("prices",ticker)
        completed = risks.weekly_risk_prep(ticker_sim,bench_returns)
        new_prices.append(completed)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
price_returns = pd.concat(new_prices)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:47<00:00, 10.30it/s]


In [7]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [8]:
classification_db.connect()
classification_simulation = classification_db.retrieve("sim")
classification_db.disconnect()

In [9]:
simulation["week"] = simulation["week"] + 1
classification_simulation["week"] = classification_simulation["week"] + 1
classification_simulation.rename(columns={"prediction":"classification_prediction"},inplace=True)

In [10]:
simulation = price_returns.merge(simulation.drop("adjclose",axis=1)[["year","week","ticker","prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose","training_years"],axis=1)[["year","week","ticker","classification_prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(tyields[["date","weekly_yield"]],on="date",how="left")

In [11]:
sim = simulation.dropna()

In [12]:
sim.sort_values(["date","ticker"],inplace=True)
sim = sim.groupby(["date","ticker","classification_prediction"]).mean().reset_index()

In [13]:
trades = []
values = [True,False]
classifications = [True,False]
ceilings = [True,False]
floors = [True,False]
parameters = []
positions = 10
strats = ["spec","rolling","window"]
for strat in strats:
    for value in values:
        for classification in classifications:
            for ceiling in ceilings:
                for floor in floors:
                    parameter = {
                                "value":value
                                 ,"classification":classification
                                 ,"ceiling":ceiling
                                 ,"floor":floor
                                ,"strat":strat
                                }
                    parameters.append(parameter)

In [14]:
sim.columns

Index(['date', 'ticker', 'classification_prediction', 'close', 'high', 'low',
       'open', 'volume', 'adjclose', 'adjhigh', 'adjlow', 'adjopen',
       'adjvolume', 'divcash', 'splitfactor', 'year', 'quarter', 'week',
       'return_2', 'return_3', 'return_4', 'day', 'weekly_return',
       'rolling_prediction', 'window_prediction', 'bench_return', 'variance',
       'market_cov', 'beta', 'prediction', 'weekly_yield'],
      dtype='object')

In [15]:
weeklies_db.connect()
weeklies_db.drop("trades")
for parameter in tqdm(parameters):
    backtest_data = sim.copy().dropna()
    strat = parameter["strat"]
    backtest_data = risks.weekly_backtest_prep(strat,backtest_data)
    b.weeklies_backtest(backtest_data.copy(),positions,parameter,start_date,end_date,weeklies_db)
weeklies_db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [10:20<00:00, 12.93s/it]
